# Анализ текста и привидение его к таблинчому формату

### Установка библиотек

In [1]:
!pip install pandas scikit-learn nltk pymupdf python-docx -q


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Импорт данных

Самые популярные форматы: txt, pdf, docs. Никто не запрещает любой текст привести к формату txt. Для примера загрузим эти форматы.

### txt

In [2]:
def read_txt(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

read_txt('text_data/0/0.txt')[:100]

'Отец мой Андрей Петрович Гринев в молодости своей служил при графе Минихе и вышел в отставку премьер'

### docs

In [3]:
from docx import Document

def read_docx(path):
    doc = Document(path)

    return "\n".join([p.text for p in doc.paragraphs])

read_docx('text_data/1/1.docx')[:100]

'Приступая к описанию недавних и столь странных событий, происшедших в нашем, доселе ничем не отличав'

### pdf

In [4]:
import fitz  # PyMuPDF

def read_pdf(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

read_pdf('text_data/2/2.pdf')[:100]

'Как ни старались люди, собравшись в одно небольшое место несколько сот тысяч, изуродовать ту \nземлю,'

Представим, нам дали набор txt файлов.

In [29]:
import pandas as pd
import glob
import os

files = []
texts = []
labels = []

classes = os.listdir('text_data/')
for target in classes:
    file_paths = glob.glob(rf'text_data/{target}/*.txt')
    for file_path in file_paths:
        files.append(file_path)
        texts.append(read_txt(file_path))
        labels.append(target)

df = pd.DataFrame(
    {
        'file_path': files,
        'text': texts,
        'label': labels
    }
)

df.head(3)

,file_path,text,label
0,text_data/0\0.txt,Отец мой Андрей Петрович Гринев в молодости св...,0
1,text_data/1\1.txt,Приступая к описанию недавних и столь странных...,1
2,text_data/2\2.txt,"Как ни старались люди, собравшись в одно небол...",2


## Базовая отчистка текста

In [30]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zа-я0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text


df['text'] = df['text'].map(clean_text)
df.head(3)

,file_path,text,label
0,text_data/0\0.txt,отец мой андрей петрович гринев в молодости св...,0
1,text_data/1\1.txt,приступая к описанию недавних и столь странных...,1
2,text_data/2\2.txt,как ни старались люди собравшись в одно неболь...,2


## Преобразование текста в числовые признаки

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=1000,
)

preprocess_text = vectorizer.fit_transform(df['text']).toarray()
preprocess_text_df = pd.DataFrame(preprocess_text, 
                            columns=[f'feature_{i}' for i in range(1000)])

total_df = pd.concat([df, preprocess_text_df], axis=1)

## Итоговый датасет

Данный датасет можно использовать для дальнейшнего анализа данных.

In [38]:
total_df.head(3)

,file_path,text,label,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999
0,text_data/0\0.txt,отец мой андрей петрович гринев в молодости св...,0,0.000000,0.000000,0.014265,0.00000,0.014265,0.014265,0.014265,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
1,text_data/1\1.txt,приступая к описанию недавних и столь странных...,1,0.021487,0.021487,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.016342,0.021487,0.00000,0.021487,0.042974,0.042974,0.00000,0.00000,0.00000,0.00000
2,text_data/2\2.txt,как ни старались люди собравшись в одно неболь...,2,0.000000,0.000000,0.000000,0.02374,0.000000,0.000000,0.000000,...,0.009027,0.000000,0.01187,0.000000,0.000000,0.000000,0.01187,0.01187,0.01187,0.01187
